# Lede Summer 2019 Project - Part 4
# Make a database of female and male Olympic all-time medallists (summer)
A LOT OF ISSUES WITH THE ORIGINAL FILE. Misspelling, different spellings, different names used, etc.

In the text file, cleaned:
* Missing/wrong data for Clas Thunberg
* Ahn Hyun-Soo (Victor AN) -- changed nationality from S.Korea to Russia.
* Eliminated duplicate row for Marit BJØRGEN and Sven Kramer

In [1]:
import requests
import pandas as pd
import re
import numpy as np
import os

import itertools

from bs4 import BeautifulSoup
from selenium import webdriver

#### LINKS:
https://www.paralympic.org/sdms4/hira/web/multiMedallists/type/summer

SUMMER:
https://stillmed.olympic.org/media/Document%20Library/OlympicOrg/Factsheets-Reference-Documents/Games/Records-and-Medals/Factsheet-Records-of-medals-at-the-Games-of-the-Olympiad.pdf#_ga=2.24637754.1432842169.1561491999-1688621044.1561491999

### Turn pdf to text

In [2]:
# Turn pdf into txt

def pdf_to_text(name):
    folder = "/Users/minami/Documents/Classwork/projects/Final-Project-Lede-Summer2019/my_work/"
    input1 = folder + name
    txt_name = name.replace(".pdf",".txt")
    output1 = folder + txt_name
    os.system("pdftotext '%s' '%s'" % (input1, output1))

In [3]:
pdf_to_text('oly_winter.pdf')

## Manually clean the text file using regex

* Standardise format of rows by adding columns, replacing dashes with commas, eliminating white space, formatting years so that they follow \d\d\d\d-\d\d\d\d format

In [4]:
cols = ['gender', 'medal_count', 'last_name', 'first_name', 'citizenship', 'event', 'years']
df = pd.read_csv('oly_winter_most.txt', sep=',', names=cols, index_col=False)
df.head(3)
df.shape

(38, 7)

In [5]:
df

,gender,medal_count,last_name,first_name,citizenship,event,years
0,Women,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,15.0,Marit,BJØRGEN,NOR,cross country,2002-2018
2,NaN,10.0,Raisa,SMETANINA,RUS,cross country,1976-1992
3,NaN,10.0,Stefania,BELMONDO,ITA,cross country,1992-2002
4,NaN,9.0,Lyubov,YEGOROVA,RUS,cross country,1992-2002
5,NaN,9.0,Uschi,DISL,GER,biathlon,1992-2006
6,NaN,9.0,Claudia,PECHSTEIN,GER,speed skating,1992-2006
7,NaN,9.0,Charlotte,KALLA,SWE,cross country,2010-2018
8,NaN,8.0,Gunda,NIEMANN-STIRNEMANN,GER,speed skating,1988-1998
9,NaN,8.0,Galina,KULAKOVA,URS,cross country,1968-1980


### Fill in values for gender

In [6]:
df['gender'] = df.gender.fillna(method='ffill')
df = df.dropna(subset = ['last_name'])
df

,gender,medal_count,last_name,first_name,citizenship,event,years
1,Women,15.0,Marit,BJØRGEN,NOR,cross country,2002-2018
2,Women,10.0,Raisa,SMETANINA,RUS,cross country,1976-1992
3,Women,10.0,Stefania,BELMONDO,ITA,cross country,1992-2002
4,Women,9.0,Lyubov,YEGOROVA,RUS,cross country,1992-2002
5,Women,9.0,Uschi,DISL,GER,biathlon,1992-2006
6,Women,9.0,Claudia,PECHSTEIN,GER,speed skating,1992-2006
7,Women,9.0,Charlotte,KALLA,SWE,cross country,2010-2018
8,Women,8.0,Gunda,NIEMANN-STIRNEMANN,GER,speed skating,1988-1998
9,Women,8.0,Galina,KULAKOVA,URS,cross country,1968-1980
10,Women,8.0,Karin,KANIA-ENKE,GDR,speed skating,1980


### Make a new column with the full name

In [7]:
df['full_name'] = (df.last_name+df.first_name).str.strip()
# df.full_name

/Users/minami/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Scrape medals data for athletes
* Make endpoints for urls
* Then clean up the endpoint for the extra dashes, Jr.s
* Make a list of endpoints over which I can interate
* Clean up the list of endpoints for athletes with a middle name or a hyphenated last names

In [8]:
df['endpoint'] = df.full_name.str.replace(' ','-')

/Users/minami/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Make a list of endpoints


In [9]:
athletes = df.endpoint.to_list()

### Clean the list

* Hyun Soo Ahn also goes by Victor An (and also changed citizenship from S.Korea to Russia)
* Special characters are replaced with regular alphabets in the url endpoints
* Some people don't have their full names as their endpoints (in the url) -- Marja-Liisa-KIRVESNIEMI-HAEMAELAEINEN, for example, has an endpoint of marja-liisa-hamalainen

In [10]:
for idx, name in enumerate(athletes):
    if name == 'Hyun-Soo-AHN-(Victor-AN)':
        athletes[idx] = 'Victor-AN'
    elif name == 'Victor-AN-(AHN-Hyun-Soo)':
        athletes[idx] = 'Victor-AN'
    elif name == 'Marit-BJØRGEN':
        athletes[idx] = 'marit-bjoergen'
    elif name == 'Ole-Einar-BJØRNDALEN':
        athletes[idx] = 'Ole-Einar-BJOERNDALEN'
    elif name == 'Andrea-EHRIG-SCHÖNE-MITSCHERLICH':
        athletes[idx] = 'andrea-mitscherlich'
    elif name == 'Gunda-NIEMANN-STIRNEMANN':
        athletes[idx] = 'gunda-kleemann'
        
    elif name == 'Marja-Liisa-KIRVESNIEMI-HAEMAELAEINEN':
        athletes[idx] = 'marja-liisa-hamalainen'
    elif name == 'Elena-VÄLBE':
        athletes[idx] = 'elena-vaelbe'
    elif name == 'Bjørn-DÆHLIE':
        athletes[idx] = 'bjorn-daehlie'
    elif name == 'Emil-SVENDSEN':
        athletes[idx] = 'emil-hegle-svendsen'
    elif name == 'Eero-MÄNTYRANTA':
        athletes[idx] = 'eero-mantyranta'
    elif name == 'Karin-KANIA-ENKE':
        athletes[idx] = 'karin-enke'
    else:
        pass
    
# athletes

### Edit the endpoints in the original dataframe so that they can be merged on endpoints later on!!!

In [11]:
df['endpoint_cleaned'] = athletes

/Users/minami/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
rows = []
for athlete in athletes:
    try:
        url = f'https://www.olympic.org/{athlete}'
        response = requests.get(url)
        doc = BeautifulSoup(response.text)
        medals = doc.find(class_='medal-box').text.strip().split( ' \n\n\n ')
        
        row = {}

        gold = int(medals[0][1:])
        silver = int(medals[1][1:])
        bronze = int(medals[2][1:])
        
        row['endpoint'] = athlete
        row['medals_gold'] = gold
        row['medals_silver'] = silver
        row['medals_bronze'] = bronze
        row['medals_total'] = gold + silver + bronze
        rows.append(row)
    except:
        print('-----')
        print('ERROR WITH', athlete)
        print('-----')    

In [13]:
len(rows)

36

In [14]:
df_medals = pd.DataFrame(rows)
df_medals.head(1)

,endpoint,medals_bronze,medals_gold,medals_silver,medals_total
0,marit-bjoergen,3,8,4,15


In [15]:
df.head(1)

,gender,medal_count,last_name,first_name,citizenship,event,years,full_name,endpoint,endpoint_cleaned
1,Women,15.0,Marit,BJØRGEN,NOR,cross country,2002-2018,Marit BJØRGEN,Marit-BJØRGEN,marit-bjoergen


In [16]:
merged = df.merge(df_medals, left_on='endpoint_cleaned', right_on='endpoint')
merged.shape

(38, 15)

In [17]:
# Check to see the total medal counts match up
merged[merged.medal_count != merged.medals_total]

,gender,medal_count,last_name,first_name,citizenship,event,years,full_name,endpoint_x,endpoint_cleaned,endpoint_y,medals_bronze,medals_gold,medals_silver,medals_total
19,Men,9.0,Sixten,JERNBERG,SWE,cross country,1956-1964,Sixten JERNBERG,Sixten-JERNBERG,Sixten-JERNBERG,Sixten-JERNBERG,1,4,3,8
30,Men,7.0,Clas,THUNBERG,FIN,speed skating,1924-1928,Clas THUNBERG,Clas-THUNBERG,Clas-THUNBERG,Clas-THUNBERG,0,2,0,2
34,Men,7.0,Veikko,HAKULINEN,FIN,cross country,1952-1964,Veikko HAKULINEN,Veikko-HAKULINEN,Veikko-HAKULINEN,Veikko-HAKULINEN,1,3,2,6


# FACT CHECK

The medal count for Jernberg should be 9. (9 medals - 4 gold, 3 silver, 2 bronze)
* http://www.espn.com/olympics/winter/2010/athletes/_/athlete/33164/sixten-jernberg

The medal count for Thunberg should be 7. (7 medals - 5 gold, 1 silver, 1 bronze)
* http://www.espn.com/olympics/winter/2010/athletes/_/athlete/32565/clas-thunberg

The medal count for HAKULINEN should be 7. 3 Gold, 3 Silver, 1 Bronze (7 Total)
* https://www.sports-reference.com/olympics/athletes/ha/veikko-hakulinen-1.html

## Manually edit incorrect values 

In [18]:
rows[19]['medals_bronze'] = 2
rows[19]['medals_total'] = 9
rows[19]

{'endpoint': 'Sixten-JERNBERG',
 'medals_gold': 4,
 'medals_silver': 3,
 'medals_bronze': 2,
 'medals_total': 9}

In [19]:
rows[28]['medals_gold'] = 5
rows[28]['medals_silver'] = 1
rows[28]['medals_bronze'] = 1
rows[28]['medals_total'] = 7
rows[28]

{'endpoint': 'Clas-THUNBERG',
 'medals_gold': 5,
 'medals_silver': 1,
 'medals_bronze': 1,
 'medals_total': 7}

In [20]:
rows[-4]['medals_silver'] = 3
rows[-4]['medals_total'] = 7
rows[-4]

{'endpoint': 'Veikko-HAKULINEN',
 'medals_gold': 3,
 'medals_silver': 3,
 'medals_bronze': 1,
 'medals_total': 7}

# Re-do the dataframe transformation so that the correct medal counts are in the final, 'merged' dataframe

In [21]:
df_medals = pd.DataFrame(rows)
df_medals.shape

(36, 5)

In [22]:
merged = df.merge(df_medals, left_on='endpoint_cleaned', right_on='endpoint')

In [23]:
merged.shape

(38, 15)

In [24]:
# Check to see the total medal counts match up
merged[merged.medal_count != merged.medals_total]

,gender,medal_count,last_name,first_name,citizenship,event,years,full_name,endpoint_x,endpoint_cleaned,endpoint_y,medals_bronze,medals_gold,medals_silver,medals_total


## Now, medal_count and medals_total match.
* Remove medal_count, endpoint_x, endpoint_y columns to avoid confusion

In [25]:
merged = merged.drop(columns=["medal_count", 'endpoint_x', 'endpoint_y'])

In [26]:
merged['gender'] = merged.gender.str.title()
merged['game_type'] = 'Olympic'
merged['season'] = 'Winter'

In [27]:
merged

,gender,last_name,first_name,citizenship,event,years,full_name,endpoint_cleaned,medals_bronze,medals_gold,medals_silver,medals_total,game_type,season
0,Women,Marit,BJØRGEN,NOR,cross country,2002-2018,Marit BJØRGEN,marit-bjoergen,3,8,4,15,Olympic,Winter
1,Women,Raisa,SMETANINA,RUS,cross country,1976-1992,Raisa SMETANINA,Raisa-SMETANINA,1,4,5,10,Olympic,Winter
2,Women,Stefania,BELMONDO,ITA,cross country,1992-2002,Stefania BELMONDO,Stefania-BELMONDO,5,2,3,10,Olympic,Winter
3,Women,Lyubov,YEGOROVA,RUS,cross country,1992-2002,Lyubov YEGOROVA,Lyubov-YEGOROVA,0,6,3,9,Olympic,Winter
4,Women,Uschi,DISL,GER,biathlon,1992-2006,Uschi DISL,Uschi-DISL,3,2,4,9,Olympic,Winter
5,Women,Claudia,PECHSTEIN,GER,speed skating,1992-2006,Claudia PECHSTEIN,Claudia-PECHSTEIN,2,5,2,9,Olympic,Winter
6,Women,Charlotte,KALLA,SWE,cross country,2010-2018,Charlotte KALLA,Charlotte-KALLA,0,3,6,9,Olympic,Winter
7,Women,Gunda,NIEMANN-STIRNEMANN,GER,speed skating,1988-1998,Gunda NIEMANN-STIRNEMANN,gunda-kleemann,1,3,4,8,Olympic,Winter
8,Women,Galina,KULAKOVA,URS,cross country,1968-1980,Galina KULAKOVA,Galina-KULAKOVA,2,4,2,8,Olympic,Winter
9,Women,Karin,KANIA-ENKE,GDR,speed skating,1980,Karin KANIA-ENKE,karin-enke,1,3,4,8,Olympic,Winter


## Add the other_info and alternate_name columns so that the columns match Paralympic dataframes

In [28]:
merged['other_info'] = ''
merged['alternate_name'] = ''

In [29]:
merged.isna().sum()

gender              0
last_name           0
first_name          0
citizenship         0
event               0
years               0
full_name           0
endpoint_cleaned    0
medals_bronze       0
medals_gold         0
medals_silver       0
medals_total        0
game_type           0
season              0
other_info          0
alternate_name      0
dtype: int64

In [30]:
merged.to_csv('oly_winter_complete.csv', index=False)

## Manually clean up the oly_winter_complete.csv 

* Add other_info and alternate_name for Victor An/Hyun-Soo Ahn
* Eliminate the duplicate rows for Victor An
* Save cleaned file as oly_winter_complete_cleaned.csv

In [34]:
df = pd.read_csv('oly_winter_complete_cleaned.csv')
df

,gender,last_name,first_name,citizenship,event,years,full_name,endpoint_cleaned,medals_bronze,medals_gold,medals_silver,medals_total,game_type,season,other_info,alternate_name
0,Women,Marit,BJØRGEN,NOR,cross country,2002-2018,Marit BJØRGEN,marit-bjoergen,3,8,4,15,Olympic,Winter,NaN,NaN
1,Women,Raisa,SMETANINA,RUS,cross country,1976-1992,Raisa SMETANINA,Raisa-SMETANINA,1,4,5,10,Olympic,Winter,NaN,NaN
2,Women,Stefania,BELMONDO,ITA,cross country,1992-2002,Stefania BELMONDO,Stefania-BELMONDO,5,2,3,10,Olympic,Winter,NaN,NaN
3,Women,Lyubov,YEGOROVA,RUS,cross country,1992-2002,Lyubov YEGOROVA,Lyubov-YEGOROVA,0,6,3,9,Olympic,Winter,NaN,NaN
4,Women,Uschi,DISL,GER,biathlon,1992-2006,Uschi DISL,Uschi-DISL,3,2,4,9,Olympic,Winter,NaN,NaN
5,Women,Claudia,PECHSTEIN,GER,speed skating,1992-2006,Claudia PECHSTEIN,Claudia-PECHSTEIN,2,5,2,9,Olympic,Winter,NaN,NaN
6,Women,Charlotte,KALLA,SWE,cross country,2010-2018,Charlotte KALLA,Charlotte-KALLA,0,3,6,9,Olympic,Winter,NaN,NaN
7,Women,Gunda,NIEMANN-STIRNEMANN,GER,speed skating,1988-1998,Gunda NIEMANN-STIRNEMANN,gunda-kleemann,1,3,4,8,Olympic,Winter,NaN,NaN
8,Women,Galina,KULAKOVA,URS,cross country,1968-1980,Galina KULAKOVA,Galina-KULAKOVA,2,4,2,8,Olympic,Winter,NaN,NaN
9,Women,Karin,KANIA-ENKE,GDR,speed skating,1980,Karin KANIA-ENKE,karin-enke,1,3,4,8,Olympic,Winter,NaN,NaN
